# Image Inpainting using Besov Priors
This notebook demonstrates the use of Besov priors for solving image inpainting problems using Bayesian inference. The workflow includes defining missing regions, setting up the forward model, and performing posterior sampling using the RTO-MH sampler.

In [ ]:
# Import required libraries and modules
from inpainting import inpainting
from besov_prior import besov_prior
from inverse_problem import inverse_problem
from rto_mh import rto_mh
import numpy as np

## Define the Test Signal
We define a piecewise smooth test signal with multiple intervals to simulate the ground truth.

In [ ]:
def Test_Signal(x):
    """
    Generate a piecewise smooth test signal.
    Args:
        x (np.ndarray): Input array of spatial points.
    Returns:
        Signal (np.ndarray): Piecewise smooth test signal.
    """
    Signal = np.zeros(len(x))
    interval_1 = np.logical_and(0.1 <= x, x <= 0.35)
    interval_2 = np.logical_and(0.35 < x, x <= 0.45)
    interval_3 = np.logical_and(0.45 < x, x <= 0.60)
    interval_4 = np.logical_and(0.60 < x, x < 0.90)
    # Define signal values for each interval
    Signal[interval_1] = np.exp(-(x[interval_1]-0.35)**2*150)
    Signal[interval_2] = 1.0
    Signal[interval_3] = 0.8
    Signal[interval_4] = 0.40
    return Signal

## Generate the Test Signal
We generate the test signal over a spatial grid

In [ ]:
# Generate the spatial grid and test signal
J = 9 # Wavelet level
n = 2**J # Number of spatial points
x = np.linspace(0, 1, 2**J, endpoint=False) # Spatial grid
signal = Test_Signal(x) # Generate the test signal

## Define Missing Regions for Inpainting
Specify the regions to be removed (missing data) and set up the inpainting problem.

In [ ]:
# Define missing regions and set up the inpainting problem
np.random.seed(5) # Set random seed for reproducibility

# Indices of missing regions
index_remove = np.concatenate((np.argwhere(np.logical_and(x>=0.2725,x<=0.3225)),np.argwhere(np.logical_and(x>=0.5725,x<=0.6225))),axis=0)[:,0]
index_remove_1 = np.argwhere(np.logical_and(x>=0.2725,x<=0.3225))[:,0]
index_remove_2 = np.argwhere(np.logical_and(x>=0.5725,x<=0.6225))[:,0]

# Indices of observed data\
index_remain = np.delete(np.linspace(0,len(x)-1,len(x),dtype=int),index_remove)

# Initialize the inpainting forward model
likelihood = inpainting(index_remove)
likelihood.set_data(signal,noise_level=2.0)

# Save relevant data for later use
m = len(likelihood.data) # Number of observed data points
np.save("Inpainting_signal.npy",signal)
np.save("Inpainting_data.npy",likelihood.data)
np.save("Inpainting_region1.npy",index_remove_1)
np.save("Inpainting_region2.npy",index_remove_2)

## Set Up the Besov Prior
We initialize the Besov prior for the inpainting problem.

In [ ]:
# Initialize the Besov prior
s = 1.2  #Smoothness parameter
p = 1.5  # Regularity parameter
wavelets = ['db1','db8'] # Wavelet types
delt = 0.025 # Regularization parameter
level = 0
prior = besov_prior(J=J,delt=delt, level=level ,s=s, p=p,wavelet=wavelets[0])

## Perform Posterior Sampling
We use the RTO-MH sampler to generate posterior samples for each wavelet type

In [ ]:
np.random.seed(5)
nsamp = 10000  # Number of samples
x0 = 10*np.ones(len(x))# Initial guess for optimization

# Loop through wavelet types
rto_sampler=rto_mh(x0,m+n,samp=nsamp)
for wavelet in wavelets:
    prior.wavelet = wavelet
    rto_sampler.x0 = np.ones(n)

    # Define the inverse problem
    problem = inverse_problem(likelihood,prior,likelihood.jac_const(n) @ prior.jac_const(n))

    # Perform RTO sampling
    zMap=rto_sampler.initialize_Q(problem)
    rto_sampler.x0=10*zMap
    chain, acc_rate, index_accept, log_c_chain  = rto_sampler.sample(problem)
    print(acc_rate)

    # Save the results for further analysis
    np.save(wavelet + str(s) + str(p) + "Inpainting_samples.npy", chain)
    np.save(wavelet + str(s) + str(p) + "Inpainting_index_accept.npy",index_accept)

    # Reset the sampler for the next wavelet type
    rto_sampler.Q = np.identity(rto_sampler.Nrand)
    rto_sampler.eps = np.zeros(n+m)

## Set Up the Besov Prior
We initialize the Besov prior for the inpainting problem with various choices of parameters

In [ ]:
# Initialize the Besov prior
s = [0.8, 1.4, 2.0] #Smoothness parameters
p = [1.0, 1.5, 2.0] # Regularity parameters
wavelet = 'db8' # Wavelet
delt = 0.025 # Regularization parameter
level = 0
prior = besov_prior(J=J,delt=delt, level=level ,s=s[0], p=p[0],wavelet=wavelet)


## Perform Posterior Sampling
We use the RTO-MH sampler to generate posterior samples for each parameter setup

In [ ]:
np.random.seed(5)
nsamp = 10000 # Number of samples
x0 = 10*np.ones(len(x)) # Initial guess for optimization
rto_sampler=rto_mh(x0,m+n,samp=nsamp)
# Loop through parameter combinations of s and p.
for i in range(len(p)):
    for j in range(len(s)):
        rto_sampler.x0 = np.ones(n)
        # Update prior parameters
        prior.p = p[i]
        prior.s = s[j]

        # Define the inverse problem
        problem = inverse_problem(likelihood,prior,likelihood.jac_const(n) @ prior.jac_const(n))

        # Inititalize the sampler
        zMap=rto_sampler.initialize_Q(problem)
        xMap = prior.transform(zMap)
        rto_sampler.x0=10*zMap

        # Perform RTO sampling
        chain, acc_rate, index_accept, log_c_chain  = rto_sampler.sample(problem)
        print(acc_rate)

        # Save the results for further analysis
        np.save(wavelet + str(s[j]) + str(p[i]) + "Inpainting_samples.npy", chain)
        np.save(wavelet + str(s[j]) + str(p[i]) + "Inpainting_index_accept.npy",index_accept)

        # Reset the sampler for the next parameter combination
        rto_sampler.Q = np.identity(rto_sampler.Nrand)
        rto_sampler.eps = np.zeros(n+m)
